# 安徽安庆市项目
## 将MEIC中本地清单的范围设为0
## `Set MEIC to 0 in Anqing`

---
*@author: Evan*\
*@date: 2023-10-05*

In [2]:
import xarray as xr
import numpy as np
import pandas as pd
import os

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append('../../src/')
from namelist import *
from integrate import *

In [3]:
ncfile = xr.open_dataset(progdir + 'GRIDCRO2D_2023141.nc')
lon = ncfile.LON.squeeze()
lat = ncfile.LAT.squeeze()

shp = gpd.read_file(progdir + 'shapefile/Anqing/Anqing.shp')

mask = polygon_to_mask(shp.geometry[0],lon,lat)
mask_da = xr.DataArray(mask,dims=('ROW','COL'))

## MEIC
依据2.2步骤中生成的LEAQ文件，生成一个全为0的nc

In [8]:
leaq = xr.open_dataset(leaq_bio_file)
leaq_zero = xr.Dataset()

for coord_name, coord in leaq.coords.items():
    leaq_zero[coord_name] = coord

for var_name, var in leaq.variables.items():
    if var_name not in leaq.coords:
        leaq_zero[var_name] = var.copy(data=np.zeros_like(var.values))
leaq_zero.to_netcdf(datadir + f'step2_meic_and_leaq/leaq_zero.ncf')

In [9]:
sections = ['ind','pow','tra','res','agr']

for sec in sections:
    ds_A = xr.open_dataset(datadir + f'step2_meic_and_leaq/leaq_zero.ncf')
    ds_B = xr.open_dataset(eval(f'meic_{sec}_file'))

    merged = merge_datasets(mask_da,ds_A,ds_B)
    merged.to_netcdf(datadir + f'step2_meic_and_leaq/emis.CN3AH_135X138.{sec}.ncf')

## LEAQ

In [11]:
meic = xr.open_dataset(meic_ind_file)
meic_zero = xr.Dataset()

for coord_name, coord in meic.coords.items():
    meic_zero[coord_name] = coord

for var_name, var in meic.variables.items():
    if var_name not in meic.coords:
        meic_zero[var_name] = var.copy(data=np.zeros_like(var.values))
meic_zero.to_netcdf(datadir + f'step2_meic_and_leaq/meic_zero.ncf')

In [12]:
sections = ['bio','bol','bld','dst','elc','rod','nro','pet','sol','oth']

for sec in sections:
    ds_A = xr.open_dataset(eval(f'leaq_{sec}_file'))
    ds_B = xr.open_dataset(datadir + f'step2_meic_and_leaq/meic_zero.ncf')

    merged = merge_datasets(mask_da,ds_A,ds_B)
    merged.to_netcdf(datadir + f'step2_meic_and_leaq/emis.CN3AH_135X138.{sec}.ncf')